# FP management

In thi notebook we will define the various methods to perform changes to the database to keep track of the various events. In particular we will take care of:

- Adding new free agent players to the database, when they are included in the FC list
- Assigning trophies
- Assigning fines
- Calculating total roster value to determine the salary, extra budget and luxury tax
- TBD
- UPDATE PLAYER NAME IF IT HAS CHANGED ON FC:
    - Sometime FC changes the name of a player. In 2019/2020 LUKAKU was 'LUKAKU R.', now just 'LUKAKU', because Jordan left serie A. We need a way of scanning the players in our db by ID and update the name as needed. Maybe it can be included in the stats update algorithm

In [26]:
import pandas as pd
import json
with open('credential.json','r') as f:
    cred = json.load(f)

## UPDATE stats

In [27]:
stats = pd.read_pickle('../IGNOBEL/Statistiche_giocatori.pkl')

In [28]:
class new_player:
    
    def __init__(self, stats, name = False, Id = False, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    
        self.par = CR, DB, CO

        self.stats = dict(stats[stats.Nome == name].T[0][4:20])
        
        if name:
            filt = name
        elif Id:
            filt = Id
        
    def update_stats(self):
        
        CR, DB, CO = self.par
        cluster = MongoClient(CR)
        db = cluster[DB]
        collection = db[CO]
        
        stats = self.stats
        
        collection.update_one(filt, {'info.stats': stats})

In [ ]:
def update_stats_all( CR = cred['cred'], DB = 'Game', CO = 'Players'):
    None